In [1]:
%load_ext ipycache
%load_ext autoreload
%autoreload 2

/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python3.6/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/global/homes/k/ktran/miniconda3/envs/gaspy/lib/python3.6/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [2]:
import sys
sys.path.append('..')

# Initialization
## Hyperparameters for discovery

In [3]:
adsorbate = 'OH'

## Gather validation data

In [4]:
%%cache example_data.pkl training_features, training_labels, training_surfaces, sampling_features, sampling_labels, sampling_surfaces

import random
import numpy as np
from gaspy.gasdb import get_adsorption_docs, get_surface_from_doc


docs = get_adsorption_docs(adsorbate)
# docs = random.sample(docs, total_size_limit)
random.shuffle(docs)


def parse_docs(docs):
    '''
    Our testing framework needs us to divide information into features, labels, and surfaces.
    This function will convert a list of docs/dictionaries into these things.
    
    Arg:
        docs    A list of dictionaries from `gaspy.gasdb.get_adsorption_docs`
    Returns:
        features    ...just the `docs` arg
        labels      A list of the 'energy' keys in `docs`
        surfaces    A list of 4-tuples indicating the surfaces of `docs
    '''
    features = docs
    labels = [doc['energy'] for doc in docs]
    surfaces = [get_surface_from_doc(doc) for doc in docs]
    return features, labels, surfaces


training_features, training_labels, training_surfaces = parse_docs(docs[:initial_training_size])
sampling_features, sampling_labels, sampling_surfaces = parse_docs(docs[initial_training_size:])

[Skipped the cell's code and loaded variables sampling_features, sampling_labels, sampling_surfaces, training_features, training_labels, training_surfaces from file '/global/cfs/cdirs/m2755/ktran/sandbox/2020/catalyst-acquisitions/notebooks/example_data.pkl'.]
successfully loaded your custom FW_config.yaml!
Now pulling adsorption documents...


4077it [00:00, 45544.97it/s]


# Fingerprint

In [11]:
%%cache fingerprints.pkl training_fingerprints, training_labels, training_surfaces, sampling_fingerprints, sampling_labels, sampling_surfaces
from gaspy_regress import fingerprinters


inner_fingerprinter = fingerprinters.InnerShellFingerprinter()
outer_fingerprinter = fingerprinters.OuterShellFingerprinter()
fingerprinter = fingerprinters.StackedFingerprinter(inner_fingerprinter, outer_fingerprinter)

training_fingerprints = fingerprinter.fit_transform(training_features)
sampling_fingerprints = fingerprinter.transform(sampling_features)

[Saved variables 'sampling_fingerprints, sampling_labels, sampling_surfaces, training_fingerprints, training_labels, training_surfaces' to file '/global/cfs/cdirs/m2755/ktran/sandbox/2020/catalyst-acquisitions/notebooks/fingerprints.pkl'.]
